IMPORTS

In [14]:
import os
import json
import csv
import time
import torch
import timm
import torch.nn as nn
from torch.nn.utils import prune
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
from typing import Tuple
from torch.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from openpyxl import Workbook, load_workbook

DCLRATIONS

In [15]:
# with open("GlobVar.json", "r") as file:
#     gv = json.load(file)

# mod_id = gv['mod_id']
# MOD_ID = mod_id
BATCH_ID = 5
MOD_ID = 2
BATCH_SIZE = 32
NUM_EPOCHS = 30
LEARNING_RATE = 1e-4
TRANS_WEIGHT = 1.5
ROTATION_WEIGHT = 1.0
ANGULAR_WEIGHT = 0.1
PATIENCE = 3
IMG_SIZE = 224


In [16]:
BASE_DIR = os.path.expanduser("~/SKRIPSI/SCRIPTS")
DATASET_DIR = os.path.join(BASE_DIR, f"dataset/batch{BATCH_ID}")
MODEL_SAVE_PATH = os.path.join(BASE_DIR, f"model/S-ConvNeXt6DP{BATCH_ID}.{MOD_ID}.pth")
BEST_MODEL_PATH = os.path.join(BASE_DIR, f"model/BEST-S-ConvNeXt6DP{BATCH_ID}.{MOD_ID}.pth")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATASETCLASS

In [17]:
class PoseDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.annotations.iloc[idx, 1:].astype(np.float32).values)
        if self.transform:
            image = self.transform(image)
        return image, label

Conversions loss functions rmse yadaydadaydada

In [18]:
def rotation_error(R_pred, R_gt):
    """Compute angular error in degrees between rotation matrices."""
    R_diff = torch.bmm(R_pred.transpose(1, 2), R_gt)
    trace = torch.diagonal(R_diff, dim1=1, dim2=2).sum(dim=1)
    eps = 1e-6
    angle_rad = torch.acos(torch.clamp((trace - 1) / 2, min=-1 + eps, max=1 - eps))
    return torch.rad2deg(angle_rad)


def geodesic_loss(R_pred, R_gt):
    R_diff = torch.bmm(R_pred.transpose(1, 2), R_gt)
    trace = torch.diagonal(R_diff, dim1=1, dim2=2).sum(dim=1)
    eps = 1e-6
    angle = torch.acos(torch.clamp((trace - 1) / 2, -1 + eps, 1 - eps))
    return angle.mean()

def compute_rotation_matrix_from_ortho6d(poses_6d):
    """Convert 6D rotation representation to 3x3 rotation matrices."""
    x_raw = poses_6d[:, 0:3]
    y_raw = poses_6d[:, 3:6]

    x = F.normalize(x_raw, dim=1)
    z = F.normalize(torch.cross(x, y_raw, dim=1), dim=1)
    y = torch.cross(z, x, dim=1)

    rot = torch.stack((x, y, z), dim=-1)  # Shape: [B, 3, 3]
    return rot

def combined_loss(output, target, trans_w=1.0, rot_w=1.0, ang_w=0.1):
    pred_trans = output[:, :3]
    gt_trans = target[:, :3]
    pred_rot_6d = output[:, 3:9]
    gt_rot_6d = target[:, 3:9]

    pred_rot = compute_rotation_matrix_from_ortho6d(pred_rot_6d)
    gt_rot = compute_rotation_matrix_from_ortho6d(gt_rot_6d)

    loss_trans = F.mse_loss(pred_trans, gt_trans)
    loss_rot = geodesic_loss(pred_rot, gt_rot)
    return trans_w * loss_trans + rot_w * loss_rot


def rotation_error_deg_from_6d(pred_6d, gt_6d):
    # Ensure same dtype (float32)
    pred_6d = pred_6d.float()
    gt_6d = gt_6d.float()

    R_pred = compute_rotation_matrix_from_ortho6d(pred_6d)
    R_gt = compute_rotation_matrix_from_ortho6d(gt_6d)

    R_diff = torch.bmm(R_pred.transpose(1, 2), R_gt)
    trace = R_diff[:, 0, 0] + R_diff[:, 1, 1] + R_diff[:, 2, 2]
    cos_theta = (trace - 1) / 2
    cos_theta = torch.clamp(cos_theta, -1.0, 1.0)

    theta = torch.acos(cos_theta)
    return torch.rad2deg(theta.mean())


def compute_errors(outputs, labels):
    outputs = outputs.float()
    labels = labels.float()

    pred_trans = outputs[:, :3]
    pred_rot_6d = outputs[:, 3:9]

    gt_trans = labels[:, :3]
    gt_rot_6d = labels[:, 3:9]

    trans_rmse = torch.sqrt(F.mse_loss(pred_trans, gt_trans))
    rot_rmse = rotation_error_deg_from_6d(pred_rot_6d, gt_rot_6d)

    return trans_rmse.item(), rot_rmse.item()


def calculate_translation_rmse(preds, gts):
    trans_rmse = np.sqrt(np.mean(np.sum((preds[:, :3] - gts[:, :3])**2, axis=1)))
    return trans_rmse * 100  # Convert m → cm

def translation_accuracy_percentage(rmse_cm, range_cm):
    return max(0.0, 100.0 * (1 - rmse_cm / range_cm))

Transform & Dataloader

In [19]:
def get_transform():
    return transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # ImageNet mean
            std=[0.229, 0.224, 0.225]    # ImageNet std
        )
    ])

def get_dataloader(split):
    base_dir = os.path.join(DATASET_DIR, split)
    csv_path = os.path.join(base_dir, "labels.csv")
    images_dir = os.path.join(base_dir, "images")  # Updated to point to the actual image folder
    dataset = PoseDataset(csv_path, images_dir, transform=get_transform())
    return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=(split == "train"))


In [20]:
def get_dataset_stats(loader):
    translations = []
    for _, labels in loader:
        translations.append(labels[:, :3])
    trans = torch.cat(translations, dim=0)
    return {
        'min': trans.min(dim=0).values,
        'max': trans.max(dim=0).values,
        'mean': trans.mean(dim=0),
        'std': trans.std(dim=0)
    }


Loading...

In [21]:
train_loader = get_dataloader("train")
val_loader = get_dataloader("val")
test_loader = get_dataloader("test")

Model Definitions

In [22]:
class ConvNeXt6DP(nn.Module):
    def __init__(self):
        super(ConvNeXt6DP, self).__init__()
        self.backbone = timm.create_model(
            "convnextv2_nano.fcmae_ft_in22k_in1k",
            pretrained=True,
            features_only=False
        )
        self.backbone.head = nn.Identity()  # Remove classifier
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling
        self.flatten = nn.Flatten()

        self.head = nn.Sequential(
            nn.Linear(640, 512),  # ✅ fix here
            nn.ReLU(),
            nn.Linear(512, 9),
            # nn.ReLU(),
            # nn.Linear(128, 9)
        )

    def forward(self, x):
        x = self.backbone.forward_features(x)  # Extract features
        x = self.pool(x)
        x = self.flatten(x)
        return self.head(x)


The setup

In [23]:
model = ConvNeXt6DP().to(DEVICE)  # Use the updated model class
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

Train func

In [24]:
def train(validate=True, resume_from_checkpoint=False):
    writer = SummaryWriter(log_dir=os.path.join(BASE_DIR, f"runs/S-ConvNeXt6DP_batch{BATCH_ID}.{MOD_ID}"))
    scaler = GradScaler()
    best_val_loss = float('inf')
    epochs_no_improve = 0
    start_epoch = 0
    now = []
    time_per_epoch = []
    # Resume from checkpoint if specified
    if resume_from_checkpoint:
        checkpoint = torch.load(MODEL_SAVE_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scaler.load_state_dict(checkpoint['scaler_state_dict'])
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        epochs_no_improve = checkpoint.get('epochs_no_improve', 0)
        start_epoch = checkpoint.get('epoch', 0)
        print(f"✅ Resumed from checkpoint at epoch {start_epoch}")
        now = [0.0] * (start_epoch + 1)

    if len(now) <= start_epoch:
        now.append(time.time())
    else:
        now[start_epoch] = time.time()

    for epoch in range(start_epoch, NUM_EPOCHS):
        print("\n")
        print(f"📦 EPOCH : {epoch + 1}")
        model.train()
        train_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}", leave=False)

        for images, labels in pbar:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad(set_to_none=True)
            with autocast(device_type="cuda"):
                outputs = model(images)
                loss = combined_loss(outputs, labels, TRANS_WEIGHT, ROTATION_WEIGHT, ANGULAR_WEIGHT)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            torch.cuda.synchronize()  # ✅ Sync after each step

            train_loss += loss.item()
            pbar.set_postfix({"Loss": loss.item()})

        avg_train_loss = train_loss / len(train_loader)
        now.append(time.time())
        time_per_epoch.append(int(now[epoch + 1] - now[epoch]))

        print(f"✅ Epoch {epoch + 1} Avg Training Loss: {avg_train_loss:.4f}")
        print(f"⏱️ Time per epoch {epoch + 1}: {time_per_epoch[epoch]}s")

        if epoch != 0 and epoch % 5 == 0:
            parameters_to_prune = [
                (module, 'weight') for module in model.modules()
                if isinstance(module, (nn.Linear, nn.Conv2d)) and hasattr(module, 'weight')
            ]
            if parameters_to_prune:
                prune.global_unstructured(
                    parameters_to_prune,
                    pruning_method=prune.L1Unstructured,
                    amount=0.1
                )
                print(f"⚠️ Pruning applied at epoch {epoch}")
            else:
                print(f"⚠️ Skipping pruning: No eligible parameters found at epoch {epoch}")

        if validate:
            model.eval()
            val_loss = 0.0
            total_trans_rmse, total_rot_rmse = 0.0, 0.0

            with torch.no_grad():
                for images, labels in val_loader:
                    images = images.to(DEVICE)
                    labels = labels.to(DEVICE)
                    with autocast(device_type="cuda"):
                        outputs = model(images)
                        loss = combined_loss(outputs, labels, TRANS_WEIGHT, ROTATION_WEIGHT, ANGULAR_WEIGHT)
                    torch.cuda.synchronize()  # ✅ Sync for accurate timing

                    val_loss += loss.item()

                    trans_rmse, rot_rmse = compute_errors(outputs, labels)
                    total_trans_rmse += trans_rmse
                    total_rot_rmse += rot_rmse

            avg_val_loss = val_loss / len(val_loader)
            avg_trans_rmse = total_trans_rmse / len(val_loader)
            avg_rot_rmse = total_rot_rmse / len(val_loader)

            print(f"📉 Validation Loss: {avg_val_loss:.4f}")
            print(f"📐 RMSE - Translation: {avg_trans_rmse:.4f}, Rotation: {avg_rot_rmse:.4f}")

            scheduler.step(avg_val_loss)

            if epoch != 0 and avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                epochs_no_improve = 0
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scaler_state_dict': scaler.state_dict(),
                    'best_val_loss': best_val_loss,
                    'epochs_no_improve': epochs_no_improve,
                    'epoch': epoch + 1
                }, BEST_MODEL_PATH)
                print(f"Model saved to: {BEST_MODEL_PATH}")
                print("💾 Best model saved.")
            else:
                epochs_no_improve += 1
                print(f"📉 No improvement ({epochs_no_improve}/{PATIENCE})")
                if epochs_no_improve >= PATIENCE:
                    print("⏹️ Early stopping triggered")
                    break

        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict(),
            'best_val_loss': best_val_loss,
            'epochs_no_improve': epochs_no_improve,
            'epoch': epoch + 1
        }, MODEL_SAVE_PATH)
        print(f"Model saved to: {MODEL_SAVE_PATH}")

    writer.close()
    return time_per_epoch


Actually training

In [25]:
time_per_epoch = train(validate=True,resume_from_checkpoint=False)



📦 EPOCH : 1


Epoch 1/30:   0%|          | 0/543 [00:00<?, ?it/s]

✅ Epoch 1 Avg Training Loss: 1.7465
⏱️ Time per epoch 1: 269s
📉 Validation Loss: 1.7518
📐 RMSE - Translation: 0.1043, Rotation: 99.4308
📉 No improvement (1/3)
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 2


✅ Epoch 2 Avg Training Loss: 1.4031
⏱️ Time per epoch 2: 289s
📉 Validation Loss: 1.1669
📐 RMSE - Translation: 0.0860, Rotation: 66.2233
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/BEST-S-ConvNeXt6DP5.2.pth
💾 Best model saved.
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 3


✅ Epoch 3 Avg Training Loss: 0.9437
⏱️ Time per epoch 3: 291s
📉 Validation Loss: 0.8077
📐 RMSE - Translation: 0.0734, Rotation: 45.8136
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/BEST-S-ConvNeXt6DP5.2.pth
💾 Best model saved.
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 4


...

✅ Epoch 26 Avg Training Loss: 0.0587
⏱️ Time per epoch 26: 289s
⚠️ Pruning applied at epoch 25
📉 Validation Loss: 0.2937
📐 RMSE - Translation: 0.0892, Rotation: 16.1416
📉 No improvement (1/3)
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 27


✅ Epoch 27 Avg Training Loss: 0.0960
⏱️ Time per epoch 27: 291s
📉 Validation Loss: 0.1110
📐 RMSE - Translation: 0.0376, Rotation: 6.2609
📉 No improvement (2/3)
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 28


✅ Epoch 28 Avg Training Loss: 0.0632
⏱️ Time per epoch 28: 290s
📉 Validation Loss: 0.1018
📐 RMSE - Translation: 0.0345, Rotation: 5.7633
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/BEST-S-ConvNeXt6DP5.2.pth
💾 Best model saved.
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 29


✅ Epoch 29 Avg Training Loss: 0.0582
⏱️ Time per epoch 29: 289s
📉 Validation Loss: 0.1015
📐 RMSE - Translation: 0.0342, Rotation: 5.7552
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/BEST-S-ConvNeXt6DP5.2.pth
💾 Best model saved.
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


📦 EPOCH : 30


✅ Epoch 30 Avg Training Loss: 0.0532
⏱️ Time per epoch 30: 291s
📉 Validation Loss: 0.0879
📐 RMSE - Translation: 0.0323, Rotation: 4.9756
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/BEST-S-ConvNeXt6DP5.2.pth
💾 Best model saved.
Model saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP5.2.pth


Update

In [26]:
# gv['mod_id'] += 1
# with open("GlobVar.json", "w") as file:
#     json.dump(gv, file, indent=4)
# print("mod_id updated in GlobVar.json")


In [27]:
CLEAN_MODEL_PATH = os.path.join(BASE_DIR, f"model/CLEAN-S-ConvNeXt6DP{BATCH_ID}.{MOD_ID}.pth")
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        if hasattr(module, "weight_orig"):
            prune.remove(module, "weight")
torch.save(model.state_dict(), CLEAN_MODEL_PATH)

Test func

In [28]:
def test_model(model, loader, mode='Test', use_amp=False):
    model.eval()
    inference_times = []
    total_loss = 0.0
    total_trans_rmse = 0.0
    total_rot_rmse = 0.0
    num_samples = 0
    all_preds, all_gts = [], []

    with torch.no_grad():
        for images, labels in tqdm(loader, desc=f"Running {mode}"):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            batch_size = images.size(0)

            torch.cuda.synchronize()
            start_time = time.time()

            if use_amp:
                with autocast(device_type="cuda"):
                    outputs = model(images)
                    loss = combined_loss(outputs, labels, TRANS_WEIGHT, ROTATION_WEIGHT, ANGULAR_WEIGHT)
            else:
                outputs = model(images)
                loss = combined_loss(outputs, labels, TRANS_WEIGHT, ROTATION_WEIGHT, ANGULAR_WEIGHT)

            torch.cuda.synchronize()
            inference_time = (time.time() - start_time) * 1000 / batch_size  # ms per image
            inference_times.append(inference_time)

            total_loss += loss.item() * batch_size
            trans_rmse, rot_rmse = compute_errors(outputs, labels)
            total_trans_rmse += trans_rmse * batch_size
            total_rot_rmse += rot_rmse * batch_size
            num_samples += batch_size

            all_preds.append(outputs.cpu().numpy())
            all_gts.append(labels.cpu().numpy())

    avg_loss = total_loss / num_samples
    avg_trans_rmse = total_trans_rmse / num_samples
    avg_rot_rmse = total_rot_rmse / num_samples
    avg_inference_time = np.mean(inference_times)

    return avg_loss, avg_trans_rmse, avg_rot_rmse, np.concatenate(all_preds), np.concatenate(all_gts), avg_inference_time


Actually testing

In [29]:
model.load_state_dict(torch.load(CLEAN_MODEL_PATH))
model.to(DEVICE)

test_avg_loss, test_avg_trans_rmse, test_avg_rot_rmse, test_preds, test_gts, test_avg_inference_time = test_model(
    model, test_loader, mode='Test', use_amp=True
)

print("\n📊 Test Summary")
print(f"🔁 Average Loss       : {test_avg_loss:.4f}")
print(f"📐 Translation RMSE   : {test_avg_trans_rmse:.4f}")
print(f"📐 Rotation RMSE      : {test_avg_rot_rmse:.4f}")
print(f"⚡ Inference Time (ms) : {test_avg_inference_time:.2f} ms/image")

Running Test: 100%|██████████| 68/68 [00:24<00:00,  2.78it/s]


📊 Test Summary
🔁 Average Loss       : 0.0875
📐 Translation RMSE   : 0.0328
📐 Rotation RMSE      : 4.9576
⚡ Inference Time (ms) : 1.10 ms/image


Val func

In [30]:
def validate_model(model_path=None):
    inference_times = []
    if model_path:
        if not os.path.exists(model_path):
            raise ValueError(f"Model path {model_path} does not exist.")
        model.load_state_dict(torch.load(model_path))
    model.to(DEVICE)
    model.eval()
    total_loss, total_trans_rmse, total_rot_rmse = 0.0, 0.0, 0.0
    num_samples = 0
    all_preds, all_gts = [], []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Running Validation"):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            batch_size = images.size(0)

            torch.cuda.synchronize()
            start_time = time.time()

            with autocast(device_type="cuda"):
                outputs = model(images)
                loss = combined_loss(outputs, labels, TRANS_WEIGHT, ROTATION_WEIGHT, ANGULAR_WEIGHT)

            torch.cuda.synchronize()
            inference_time = (time.time() - start_time) * 1000 / batch_size  # ms per image
            inference_times.append(inference_time)

            total_loss += loss.item() * batch_size
            trans_rmse, rot_rmse = compute_errors(outputs, labels)
            total_rot_rmse += rot_rmse * batch_size
            total_trans_rmse += trans_rmse * batch_size
            num_samples += batch_size

            all_preds.append(outputs.cpu().numpy())
            all_gts.append(labels.cpu().numpy())

    avg_inference_time = np.mean(inference_times)
    avg_loss = total_loss / num_samples
    avg_trans_rmse = total_trans_rmse / num_samples
    avg_rot_rmse = total_rot_rmse / num_samples

    return avg_loss, avg_trans_rmse, avg_rot_rmse, np.concatenate(all_preds), np.concatenate(all_gts), avg_inference_time


Actually validating

In [31]:
val_avg_loss, val_avg_trans_rmse, val_avg_rot_rmse, val_preds, val_gts, val_avg_inference_time = validate_model(CLEAN_MODEL_PATH)

Running Validation: 100%|██████████| 68/68 [00:24<00:00,  2.80it/s]


In [32]:
torch.cuda.empty_cache()

Calculating 

In [33]:
test_translation_rmse_cm = calculate_translation_rmse(test_preds, test_gts)
val_translation_rmse_cm = calculate_translation_rmse(val_preds, val_gts)
test_rot_accuracy = rotation_error_deg_from_6d(torch.tensor(test_preds[:, 3:]), torch.tensor(test_gts[:, 3:]))
val_rot_accuracy = rotation_error_deg_from_6d(torch.tensor(val_preds[:, 3:]), torch.tensor(val_gts[:, 3:]))

train_trans_stats = get_dataset_stats(train_loader)
val_trans_stats = get_dataset_stats(val_loader)
test_trans_stats = get_dataset_stats(test_loader)

test_range_cm = (test_trans_stats['max'].mean() - test_trans_stats['min'].mean()) * 100
val_range_cm = (val_trans_stats['max'].mean() - val_trans_stats['min'].mean()) * 100

test_trans_accuracy_pct = translation_accuracy_percentage(test_translation_rmse_cm, test_range_cm).item()
val_trans_accuracy_pct = translation_accuracy_percentage(val_translation_rmse_cm, val_range_cm).item()

test_rot_accuracy_pct = 100*(1-(test_rot_accuracy.item()/360))
val_rot_accuracy_pct = 100*(1-(val_rot_accuracy.item()/360))

Write MD

In [34]:
eval_path = os.path.join(BASE_DIR, f"model/S-ConvNeXt6DP_batch{BATCH_ID}.{MOD_ID}.md")
eval_content = f"""# Evaluation Results - Batch {BATCH_ID} - Model {MOD_ID}

## Training Configuration
- Batch Size: {BATCH_SIZE}
- Epochs: {NUM_EPOCHS}
- Learning Rate: {LEARNING_RATE}
- Translation Weight : {TRANS_WEIGHT}
- Rotation Weight : {ROTATION_WEIGHT}
- Angular Weight : {ANGULAR_WEIGHT}
- Patience : {PATIENCE}
- Image Size: {IMG_SIZE}
- Device: {DEVICE}
- Optimizer : Adam

## Model Architecture
- Backbone: Using ConvNeXtV2 Nano @ {IMG_SIZE}
- Head: Linear(620 -> 512 -> 9)

## Evaluation Metrics

### Test Set
- Average Loss: {test_avg_loss:.4f}
- Translation RMSE: {test_avg_trans_rmse:.4f}
- Translation Accuracy: {test_translation_rmse_cm:.2f} cm
- Translation Accuracy %: {test_trans_accuracy_pct:.2f}%
- Rotation RMSE: {test_avg_rot_rmse :.4f}
- Rotation Accuracy: {test_rot_accuracy:.2f}°
- Rotation Accuracy % : {test_rot_accuracy_pct:.2f} %
- Inference Speed: {test_avg_inference_time:.2f} ms/frame

### Validation Set
- Average Loss: {val_avg_loss:.4f}
- Translation RMSE: {val_avg_trans_rmse :.4f}
- Translation Accuracy: {val_translation_rmse_cm:.2f} cm
- Translation Accuracy %: {val_trans_accuracy_pct:.2f}%
- Rotation RMSE: {val_avg_rot_rmse :.4f}
- Rotation Accuracy: {val_rot_accuracy:.2f}°
- Rotation Accuracy % : {val_rot_accuracy_pct:.2f} %
- Inference Speed: {val_avg_inference_time:.2f} ms/frame

## Dataset Statistics
### Training Set
- Translation range: [{train_trans_stats['min'].mean():.2f}, {train_trans_stats['max'].mean():.2f}] m

### Validation Set
- Translation range: [{val_trans_stats['min'].mean():.2f}, {val_trans_stats['max'].mean():.2f}] m

### Test Set
- Translation range: [{test_trans_stats['min'].mean():.2f}, {test_trans_stats['max'].mean():.2f}] m

## File Locations
- Dataset Directory: {DATASET_DIR}
- Model Save Path: {MODEL_SAVE_PATH}
"""

with open(eval_path, 'w') as f:
    f.write(eval_content)
print(f"Evaluation report saved to: {eval_path}")

Evaluation report saved to: /home/moreno/SKRIPSI/SCRIPTS/model/S-ConvNeXt6DP_batch5.2.md


In [35]:
csv_path = os.path.join(BASE_DIR, "model/eval_results.csv")
write_header = not os.path.exists(csv_path)

csv_data = {
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_id': BATCH_ID,
    'model_id': MOD_ID,
    'batch_size': BATCH_SIZE,
    'epochs': NUM_EPOCHS,
    'learning_rate': LEARNING_RATE,
    'translation_weight':TRANS_WEIGHT,
    'rotation_weight' : ROTATION_WEIGHT,
    'angular_weight' : ANGULAR_WEIGHT,
    'patience' : PATIENCE,
    'test_loss': test_avg_loss,
    'test_translation_rmse': test_avg_trans_rmse,
    'test_translation_accuracy_pct': test_trans_accuracy_pct,
    'test_rotation_rmse': test_avg_rot_rmse ,
    'test_rotation_accuracy_pct':test_rot_accuracy_pct,
    'test_inference_time_ms': test_avg_inference_time,
    'validation_loss': val_avg_loss,
    'validation_translation_rmse': val_avg_trans_rmse ,
    'validation_translation_accuracy_pct': val_trans_accuracy_pct,
    'validation_rotation_rmse': val_avg_rot_rmse ,
    'validation_rotation_accuracy_pct':val_rot_accuracy_pct,
    'validation_inference_time_ms': val_avg_inference_time,
    'model_path': MODEL_SAVE_PATH,
    'eval_path' : eval_path
}

with open(csv_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_data.keys())
    if write_header:
        writer.writeheader()
    writer.writerow(csv_data)
print(f"Results appended to CSV: {csv_path}")

Results appended to CSV: /home/moreno/SKRIPSI/SCRIPTS/model/eval_results.csv


In [36]:
def safe(val):
    if isinstance(val, (np.floating, np.integer)):
        return val.item()
    return val

def log_eval_to_xlsx(
    variant="VANILA",
    head_arch="640 → 512 → 9",
    param_count=332288,
    notes="""sanity check on 51 because changed the RMSE calculations considerably also changed epoch to 30 to see if there's some more performance to be taken from, changed from 385 to 224
            JUST LET IT RUN IT'S COURSE
            """,
    xlsx_path=os.path.join(BASE_DIR, "model/EVAL.xlsx")
):
    headers = [
        "ID", "IMG_SIZE", "VARIANT", "HEAD_ARCH", "PARAM_COUNT", "BATCH_SIZE", "EPC", "DTS_ID", "DTS_LEN", "LR_RT",
        "TRAIN_TIME", "VRAM_USG", "VAL_LOSS", "TS_LOSS",
        "VAL_TRANS_RSME", "TS_TRANS_RMSE", "VAL_ROT_RSME", "TS_ROT_RMSE",
        "VAL_TRANS_ACC", "TS_TRANS_ACC", "VAL_ROT_ACC", "TS_ROT_ACC",
        "VAL_INF_MS", "TS_INF_MS", "Notes"
    ]

    model_id_str = f"{BATCH_ID}{MOD_ID}"
    dataset_len = len(train_loader.dataset) + len(val_loader.dataset) + len(test_loader.dataset)

    row = [
        int(model_id_str), IMG_SIZE, variant, head_arch, param_count, BATCH_SIZE, NUM_EPOCHS,
        BATCH_ID, dataset_len, str(LEARNING_RATE),  # Still keep LR as string
        round(sum(time_per_epoch)/60,1), 2.7,  # TRAIN_TIME, VRAM_USG

        round(val_avg_loss, 4), round(test_avg_loss, 4),
        round(val_avg_trans_rmse , 4),
        round(test_avg_trans_rmse, 4),
        round(val_avg_rot_rmse , 4),
        round(test_avg_rot_rmse , 4),

        round(val_trans_accuracy_pct, 2), round(test_trans_accuracy_pct, 2),
        round(val_rot_accuracy_pct, 2), round(test_rot_accuracy_pct, 2),

        round(val_avg_inference_time, 2), round(test_avg_inference_time, 2),

        notes
    ]


    row = [safe(x) for x in row]

    # Load or create workbook
    if os.path.exists(xlsx_path):
        wb = load_workbook(xlsx_path)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = "ConvNeXt V2 Nano"
        ws.append(headers)

    ws.append(row)
    wb.save(xlsx_path)
    print(f"✅ Evaluation logged to Excel: {xlsx_path}")


log_eval_to_xlsx()

✅ Evaluation logged to Excel: /home/moreno/SKRIPSI/SCRIPTS/model/EVAL.xlsx
